### Download current Checkin CSV from the [checkin website](https://msc-registration-checkin.onrender.com/)

In [27]:
import pandas as pd

df = pd.read_csv('./MSC-Checkin.csv')

In [28]:
# Create new column mappings
df = df.rename(columns={
    'Contact First Name': 'ContactName.first',
    'Contact Last Name': 'ContactName.last', 
    'First Name': 'Participants.name.first',
    'Last Name': 'Participants.name.last',
    'Gender': 'Participants.gender'
})

# Reorder columns
df = df[['Family ID', 'ContactName.first', 'ContactName.last', 'Email', 'Center', 'City', 
         'Participants.name.first', 'Participants.name.last', 'Participants.gender',
         'Category', 'PAID', 'Decided', 'checkin']]

# Save to new CSV
df.to_csv('./MSC-Checkin.csv', index=False)


### Total # of People + Families Checked In

In [29]:
# Count checked in people
checked_in = df[df['checkin'] == 'checked-in']
num_checked_in = len(checked_in)

print(f"Number of people checked in: {num_checked_in}")
print("\nFamily IDs of checked in people:")
print(checked_in['Family ID'].unique())
print("\nNumber of unique families checked in:")
print(len(checked_in['Family ID'].unique()))

Number of people checked in: 555

Family IDs of checked in people:
[ 32  37   1   2   3  10  31  33  42  45  55  59  64  66  67  69  71  72
  73  74  75  76  77  78  79  80  81  83  84  85  86  87  89  91  92  95
  97  98  99 100 103 104 106 107 114 115 116 117 118 123 125 126 127 128
 129 131 139 140 141 142 143 144 145 147 149 151 152 153 154 155 156 161
 163 164 169 175 181 182 183 184 186 189 191 199 210 211 214 215 216 217
 218 221 222 225 227 230 234 239 240 244 245 248 251 255 260 261 262 263
 265 267 268 269 271 275 276 277 278 281 282 286 289 293 294 295 296 297
 301 303 306 308 309 313 316 317 318 323 327 329 330 332 334 335 337 339
 344 345 350 359 360 365 368 369 370 373 374 375 376 377 379 380 381 383
 384 386 387 389 390 391 393 394 395 396 397 398 399 401 402 411 418 433
 434 435 440 441 442 443 444 463 464 465 466 468 470 472 473 476 477 478
 479 483 484 486 488 489 494 495 501 505 507 510 512 520 521 523 525 528
 532 540 541 546 547 554 561 583 589 596 600 606 608 611 

### Checkin Count by Gender

In [30]:
# Get gender counts
gender_counts = checked_in['Participants.gender'].value_counts()
num_males = gender_counts.get('M', 0)
num_females = gender_counts.get('F', 0)

# Get adult count
num_adults = len(checked_in[checked_in['Category'] == 'Adult'])

# Get counts by grade/age
grade_counts = checked_in[checked_in['Category'].astype(str).str.isnumeric()]['Category'].value_counts().sort_index()

print(f"\nChecked-in Statistics:")
print(f"Total checked in: {num_checked_in}")
print(f"Males checked in: {num_males}")
print(f"Females checked in: {num_females}")
print(f"Adults checked in: {num_adults}")
# Define grade order
grade_order = ['Infant/ShishuVihar', 'Pre-KG', 'KG'] + [str(i) for i in range(1,13)]

# Convert grades to categorical with custom order
grade_counts = checked_in[checked_in['Category'].astype(str).str.isnumeric()]['Category']
grade_counts = grade_counts.astype('category').cat.set_categories(grade_order, ordered=True)
grade_counts = grade_counts.value_counts().sort_index()

print("\nChildren checked in by grade/age:")
print(grade_counts.to_string())



Checked-in Statistics:
Total checked in: 555
Males checked in: 244
Females checked in: 311
Adults checked in: 474

Children checked in by grade/age:
Infant/ShishuVihar     0
Pre-KG                 0
KG                     0
1                      1
2                      3
3                      2
4                      8
5                      3
6                      7
7                      8
8                     13
9                     13
10                     9
11                     7
12                     2


### Delete old `checkin_statistics.xlsx` file

In [31]:
# Delete checkin_statistics.xlsx if it exists
import os
if os.path.exists('checkin_statistics.xlsx'):
    os.remove('checkin_statistics.xlsx')


### Update with new `checkin_statistics.xlsx` file

In [32]:
import pandas as pd

# === Sheet 1: Checked In Participants ===
checked_in = df[df['checkin'].notna() & (df['checkin'] != '')]
num_checked_in = len(checked_in)

gender_counts = checked_in['Participants.gender'].value_counts()
num_males = gender_counts.get('M', 0)
num_females = gender_counts.get('F', 0)

num_adults = len(checked_in[checked_in['Category'] == 'Adult'])
adult_males = len(checked_in[(checked_in['Category'] == 'Adult') & (checked_in['Participants.gender'] == 'M')])
adult_females = len(checked_in[(checked_in['Category'] == 'Adult') & (checked_in['Participants.gender'] == 'F')])

grade_order = ['Infant/ShishuVihar', 'Pre-KG', 'KG'] + [str(i) for i in range(1, 13)]
grade_data_checked = checked_in[checked_in['Category'].isin(grade_order)]
grade_counts = grade_data_checked['Category'].astype('category')
grade_counts = grade_counts.cat.set_categories(grade_order, ordered=True)
grade_counts = grade_counts.value_counts().sort_index()

grade_gender_counts = {}
for grade in grade_counts.index:
    grade_gender_counts[grade] = {
        'M': len(checked_in[(checked_in['Category'] == grade) & (checked_in['Participants.gender'] == 'M')]),
        'F': len(checked_in[(checked_in['Category'] == grade) & (checked_in['Participants.gender'] == 'F')])
    }

summary_stats_checked_in = pd.DataFrame({
    'Metric': ['Total Checked In', 'Males Checked In', 'Females Checked In', 'Adults Checked In'] +
              [f'Grade {grade}' for grade in grade_counts.index],
    'Male': [num_males, num_males, 0, adult_males] +
            [grade_gender_counts[grade]['M'] for grade in grade_counts.index],
    'Female': [num_females, 0, num_females, adult_females] +
              [grade_gender_counts[grade]['F'] for grade in grade_counts.index],
    'Count': [num_checked_in, num_males, num_females, num_adults] +
             list(grade_counts.values)
})

# === Sheet 2: Not Checked In Participants ===
not_checked_in = df[(df['checkin'] == '') | (df['checkin'].isna())]
num_not_checked_in = len(not_checked_in)

gender_counts_nc = not_checked_in['Participants.gender'].value_counts()
num_males_nc = gender_counts_nc.get('M', 0)
num_females_nc = gender_counts_nc.get('F', 0)

num_adults_nc = len(not_checked_in[not_checked_in['Category'] == 'Adult'])
adult_males_nc = len(not_checked_in[(not_checked_in['Category'] == 'Adult') & (not_checked_in['Participants.gender'] == 'M')])
adult_females_nc = len(not_checked_in[(not_checked_in['Category'] == 'Adult') & (not_checked_in['Participants.gender'] == 'F')])

grade_data_nc = not_checked_in[not_checked_in['Category'].isin(grade_order)]
grade_counts_nc = grade_data_nc['Category'].astype('category')
grade_counts_nc = grade_counts_nc.cat.set_categories(grade_order, ordered=True)
grade_counts_nc = grade_counts_nc.value_counts().sort_index()

grade_gender_counts_nc = {}
for grade in grade_counts_nc.index:
    grade_gender_counts_nc[grade] = {
        'M': len(not_checked_in[(not_checked_in['Category'] == grade) & (not_checked_in['Participants.gender'] == 'M')]),
        'F': len(not_checked_in[(not_checked_in['Category'] == grade) & (not_checked_in['Participants.gender'] == 'F')])
    }

summary_stats_not_checked = pd.DataFrame({
    'Metric': ['Total Not Checked In', 'Males Not Checked In', 'Females Not Checked In', 'Adults Not Checked In'] +
              [f'Grade {grade}' for grade in grade_counts_nc.index],
    'Male': [num_males_nc, num_males_nc, 0, adult_males_nc] +
            [grade_gender_counts_nc[grade]['M'] for grade in grade_counts_nc.index],
    'Female': [num_females_nc, 0, num_females_nc, adult_females_nc] +
              [grade_gender_counts_nc[grade]['F'] for grade in grade_counts_nc.index],
    'Count': [num_not_checked_in, num_males_nc, num_females_nc, num_adults_nc] +
             list(grade_counts_nc.values)
})

# === Export both to the same Excel file ===
with pd.ExcelWriter('checkin_statistics.xlsx') as writer:
    summary_stats_checked_in.to_excel(writer, sheet_name='Statistics', index=False)
    summary_stats_not_checked.to_excel(writer, sheet_name='Not Checked In', index=False)

print("Both sheets exported to checkin_statistics.xlsx")


Both sheets exported to checkin_statistics.xlsx


In [33]:
# Calculate check-in percentage
total_registered = len(df)  # Total number of registered participants
total_checked_in = len(checked_in)  # Total number of checked-in participants
checkin_percentage = (total_checked_in / total_registered) * 100

print(f"\nCheck-in Statistics:")
print(f"Total Registered: {total_registered}")
print(f"Total Checked In: {total_checked_in}")
print(f"Check-in Percentage: {checkin_percentage:.1f}%")

# Add to summary stats
checkin_stats = pd.DataFrame({
    'Metric': ['Check-in Percentage'],
    'Count': [f"{checkin_percentage:.1f}%"],
    'Male': ['--'],
    'Female': ['--']
})

summary_stats = pd.concat([summary_stats, checkin_stats], ignore_index=True)

# Update Excel with new stats
with pd.ExcelWriter('checkin_statistics.xlsx') as writer:
    summary_stats.to_excel(writer, sheet_name='Statistics', index=False)



Check-in Statistics:
Total Registered: 921
Total Checked In: 555
Check-in Percentage: 60.3%


In [34]:
# Calculate San Jose center check-in stats
san_jose_df = df[df['Center'] == 'cm_San_Jose']
san_jose_not_checked = san_jose_df[~san_jose_df['checkin'].str.contains('checked-in', na=False)]

num_san_jose_total = len(san_jose_df)
num_san_jose_not_checked = len(san_jose_not_checked)

print(f"\nSan Jose Center Statistics:")
print(f"Total Registered: {num_san_jose_total}")
print(f"Not Checked In: {num_san_jose_not_checked}")
print(f"Check-in Rate: {((num_san_jose_total - num_san_jose_not_checked) / num_san_jose_total * 100):.1f}%")

# Add to summary stats
san_jose_stats = pd.DataFrame({
    'Metric': ['San Jose - Not Checked In'],
    'Count': [num_san_jose_not_checked],
    'Male': ['--'],
    'Female': ['--']
})

summary_stats = pd.concat([summary_stats, san_jose_stats], ignore_index=True)

# Update Excel with new stats
with pd.ExcelWriter('checkin_statistics.xlsx') as writer:
    summary_stats.to_excel(writer, sheet_name='Statistics', index=False)



San Jose Center Statistics:
Total Registered: 596
Not Checked In: 47
Check-in Rate: 92.1%
